In [1]:
!pip install "ray[air]==2.4.0" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 1.1 

In [1]:
import ray
from ray.air import session
from ray.air.config import DatasetConfig, RunConfig, ScalingConfig
from ray.data.preprocessors import BatchMapper
from ray.train.torch import TorchTrainer

In [2]:
# Initialize Ray
if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-05-01 14:44:44,687	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.1
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8266


In [3]:
ray.cluster_resources()

{'object_store_memory': 2147483648.0,
 'node:127.0.0.1': 1.0,
 'memory': 42867132007.0,
 'CPU': 10.0}

In [6]:
# Settings
num_cpu_workers = 6
num_gpu_workers = 0
resources_per_worker = {"CPU": 1}
use_gpu = False
device = "cpu"

In [7]:
# Preserve order
ray.data.DatasetContext.get_current().execution_options.preserve_order = True

In [ ]:
# Datasets
train_ds = ray.data.range(100)
val_ds = ray.data.range(100)
print ("train_ds:", train_ds.take())
print ("val_ds:", val_ds.take())

In [54]:
def train_loop_per_worker(config):

    # Get datasets
    train_ds = session.get_dataset_shard("train")
    val_ds = session.get_dataset_shard("val")
    
    print ("train", next(train_ds.iter_batches(batch_size=1))["value"])
    print ("val", next(val_ds.iter_batches(batch_size=1))["value"])

In [55]:
# Scaling config
scaling_config = ScalingConfig(
    num_workers=num_cpu_workers,
    resources_per_worker=resources_per_worker,
    use_gpu=False,
    _max_cpu_fraction_per_node=0.8,
)

In [57]:
# Dummy preprocessor
def udf(df):
    return df
bm = BatchMapper(udf, batch_format="pandas")

Running the cell below will produce different outputs for the first sample in train dataset every time. This is because `split=True` (default) in the `DatasetConfig`. You can run this cell multiple times to see the different value it produces each time.

**NOTE**: Ray train logger on notebooks sometimes shows the output from the previous .fit() so just keep rerunning and you'll see the values keep changing.

In [72]:
# Split = True
dataset_config = {
    "train": DatasetConfig(split=True, randomize_block_order=False),
    "val": DatasetConfig(randomize_block_order=False),
}
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    scaling_config=scaling_config,
    datasets={"train": train_ds, "val": val_ds},
    dataset_config=dataset_config,
    preprocessor=bm
)
trainer.fit()

(RayTrainWorker pid=81776) 2023-05-01 14:55:39,220	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=6]
(RayTrainWorker pid=81729) 2023-05-01 14:55:23,696	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadRange->BatchMapper] [repeated 7x across cluster]
(RayTrainWorker pid=81729) 2023-05-01 14:55:23,696	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False) [repeated 6x across cluster]
(TorchTrainer pid=81771) 2023-05-01 14:55:40,279	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=True, actor_locality_enabled=True, verbose_progress=False)


(pid=81771) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

(RayTrainWorker pid=81728) train 0    75 [repeated 6x across cluster]
(RayTrainWorker pid=81728) Name: value, dtype: int64 [repeated 12x across cluster]
(RayTrainWorker pid=81728) val 0    0 [repeated 6x across cluster]


(pid=81776) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

(pid=81779) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

...

(pid=81778) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

(pid=81777) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

Trial TorchTrainer_e696f_00000 completed. Last result: 


2023-05-01 14:55:44,292	INFO tune.py:945 -- Total run time: 9.18 seconds (9.17 seconds for the tuning loop).


Result(
  metrics={'trial_id': 'e696f_00000', 'done': True},
  path='/Users/goku/ray_results/TorchTrainer_2023-05-01_14-55-35/TorchTrainer_e696f_00000_0_2023-05-01_14-55-35',
  checkpoint=None
)

Now setting `split=False` in the `DatasetConfig` for the train dataset allows us to achieve reproducibility everytime. 

**NOTE**: Ray train logger on notebooks sometimes shows the output from the previous .fit() so the first time you run this, you'll probably see the output from the previous .fit() (where `split=True`) so just keep running it and you'll see that the train print's values never change.

In [75]:
# Split = False
dataset_config = {
    "train": DatasetConfig(split=False, randomize_block_order=False),
    "val": DatasetConfig(randomize_block_order=False),
}
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    scaling_config=scaling_config,
    datasets={"train": train_ds, "val": val_ds},
    dataset_config=dataset_config,
    preprocessor=bm
)
trainer.fit()

(RayTrainWorker pid=81893) 2023-05-01 14:56:16,321	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=6]
(RayTrainWorker pid=81862) 2023-05-01 14:56:09,513	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadRange->BatchMapper] [repeated 12x across cluster]
(RayTrainWorker pid=81862) 2023-05-01 14:56:09,513	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False) [repeated 12x across cluster]


(pid=81897) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

(RayTrainWorker pid=81859) train 0    0 [repeated 6x across cluster]
(RayTrainWorker pid=81859) Name: value, dtype: int64 [repeated 12x across cluster]
(RayTrainWorker pid=81859) val 0    0 [repeated 6x across cluster]


(pid=81898) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

(pid=81896) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

...

(pid=81896) Running 0:   0%|          | 0/20 [00:00<?, ?it/s]

Trial TorchTrainer_fcafa_00000 completed. Last result: 


2023-05-01 14:56:21,203	INFO tune.py:945 -- Total run time: 9.01 seconds (9.00 seconds for the tuning loop).


Result(
  metrics={'trial_id': 'fcafa_00000', 'done': True},
  path='/Users/goku/ray_results/TorchTrainer_2023-05-01_14-56-12/TorchTrainer_fcafa_00000_0_2023-05-01_14-56-12',
  checkpoint=None
)

2023-05-01 16:30:04,941	WARNING worker.py:1986 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/Users/goku/Documents/mwml-playground/venv/lib/python3.9/site-packages/ray/autoscaler/_private/monitor.py", line 674, in <module>
    monitor.run()
  File "/Users/goku/Documents/mwml-playground/venv/lib/python3.9/site-packages/ray/autoscaler/_private/monitor.py", line 550, in run
    self._run()
  File "/Users/goku/Documents/mwml-playground/venv/lib/python3.9/site-packages/ray/autoscaler/_private/monitor.py", line 454, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)

[2023-05-01 16:30:05,042 E 80092 1715039] core_worker.cc:569: :info_message: Attempting to recover 118 lost objects by resubmitting their tasks. To disable object reconstruction, set @ray.remote(max_retries=0).
